In [112]:
import geopandas as gpd
import pandas as pd
import parquet

In [113]:
wells = pd.read_csv("raw_data/cleaned_well_data.csv")
wells['geometry'] = gpd.points_from_xy(
    x=wells.SITE_LONGIT_DECIMAL, 
    y=wells.SITE_LATITUDE_DECIMAL, 
    crs="EPSG:4326"
    )
wells = gpd.GeoDataFrame(data=wells, geometry='geometry')
# wells = pd.read_csv("raw_data/Well_Registry.csv")
wells['date'] = pd.to_datetime(wells['date']) 
wells.info()

/tmp/ipykernel_13634/3837188658.py:1: DtypeWarning: Columns (36,45) have mixed types. Specify dtype option on import or set low_memory=False.
  wells = pd.read_csv("raw_data/cleaned_well_data.csv")


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 289301 entries, 0 to 289300
Data columns (total 57 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Unnamed: 0                  289301 non-null  int64         
 1   wellid                      289301 non-null  int64         
 2   WLWA_ID                     289301 non-null  int64         
 3   date                        288969 non-null  datetime64[ns]
 4   depth                       275285 non-null  float64       
 5   WLWA_WATER_LEVEL_ELEVATION  275285 non-null  float64       
 6   WLWA_SOURCE_CODE            283252 non-null  object        
 7   WLWA_METHOD_CODE            272560 non-null  object        
 8   WLWA_REMARK_CODE            24846 non-null   object        
 9   SITE_WELL_ALTITUDE          289301 non-null  float64       
 10  SITE_LOCAL_ID               289301 non-null  object        
 11  SITE_LATIT_DEGREE           289

In [114]:
wells = wells[wells["date"]>="1/1/2000"]
wells = wells[wells["date"]<="1/1/2020"]
wells["utc_seconds"] = pd.to_datetime(wells['date']).astype(int)/ 10**9

In [115]:
wells = wells[wells["depth"].notnull()]
wells.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 82063 entries, 417 to 289300
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  82063 non-null  int64         
 1   wellid                      82063 non-null  int64         
 2   WLWA_ID                     82063 non-null  int64         
 3   date                        82063 non-null  datetime64[ns]
 4   depth                       82063 non-null  float64       
 5   WLWA_WATER_LEVEL_ELEVATION  82063 non-null  float64       
 6   WLWA_SOURCE_CODE            82062 non-null  object        
 7   WLWA_METHOD_CODE            81991 non-null  object        
 8   WLWA_REMARK_CODE            5634 non-null   object        
 9   SITE_WELL_ALTITUDE          82063 non-null  float64       
 10  SITE_LOCAL_ID               82063 non-null  object        
 11  SITE_LATIT_DEGREE           82063 non-null  int6

In [116]:
well1 = wells[wells["wellid"]==343957112322001]

In [117]:
import numpy as np
well1
df_unix_sec = pd.to_datetime(well1['date']).astype(int)/ 10**9
f = np.polyfit(well1["utc_seconds"], well1['depth'], deg=1)

# Slope
f[0]*(60*60*24*365*20)

3.7833837261255407

In [118]:
def calculate_trend_slope(df):
    f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
    # Slope
    return f[0]*(60*60*24*365*20)


In [119]:
wells = wells.groupby('wellid').agg({
    'geometry':'first',
    'depth': lambda x: list(x),
    'utc_seconds': lambda x: list(x)
    }).reset_index()

In [120]:
wells

,wellid,geometry,depth,utc_seconds
0,3314281114620,POINT (-111.77217 33.24114),[47.2],[1207699200.0]
1,33131111532801,POINT (-111.89111 33.22514),[79.8],[1208736000.0]
2,33253111231901,POINT (-112.51925 33.42536),[165.0],[1040342400.0]
3,35442109265701,POINT (-109.46589 35.73914),[177.0],[1098403200.0]
4,311958110562701,POINT (-110.94114 31.33292),"[12.9, 13.15, 12.57, 11.9, 12.55, 12.46, 12.12...","[1213056000.0, 1179273600.0, 1280275200.0, 128..."
...,...,...,...,...
14973,370058110175801,POINT (-110.29938 37.01621),"[11.0, 11.0, 10.0, 12.0, 12.0, 12.0, 11.0, 11....","[1001030400.0, 1007683200.0, 1016668800.0, 102..."
14974,370119110181401,POINT (-110.30404 37.02159),"[14.0, 14.0, 16.0, 15.4, 14.0, 15.0, 14.0, 14....","[949536000.0, 956016000.0, 966816000.0, 977270..."
14975,370120113555201,POINT (-113.93108 37.02164),[765.4],[1110844800.0]
14976,370211110185901,POINT (-110.31642 37.03642),"[13.0, 14.0, 14.0, 14.0, 15.1, 14.0, 14.0, 12....","[949536000.0, 956016000.0, 966816000.0, 966816..."


In [121]:
wells["depth_20_year_trend"] = wells.apply(calculate_trend_slope, axis=1)
wells

/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykernel_13634/4240600141.py:2: RankWarning: Polyfit may be poorly conditioned
  f = np.polyfit(df["utc_seconds"], df['depth'], deg=1)
/tmp/ipykerne

,wellid,geometry,depth,utc_seconds,depth_20_year_trend
0,3314281114620,POINT (-111.77217 33.24114),[47.2],[1207699200.0],12.325082
1,33131111532801,POINT (-111.89111 33.22514),[79.8],[1208736000.0],20.819871
2,33253111231901,POINT (-112.51925 33.42536),[165.0],[1040342400.0],50.016610
3,35442109265701,POINT (-109.46589 35.73914),[177.0],[1098403200.0],50.818060
4,311958110562701,POINT (-110.94114 31.33292),"[12.9, 13.15, 12.57, 11.9, 12.55, 12.46, 12.12...","[1213056000.0, 1179273600.0, 1280275200.0, 128...",-3.808731
...,...,...,...,...,...
14973,370058110175801,POINT (-110.29938 37.01621),"[11.0, 11.0, 10.0, 12.0, 12.0, 12.0, 11.0, 11....","[1001030400.0, 1007683200.0, 1016668800.0, 102...",11.122620
14974,370119110181401,POINT (-110.30404 37.02159),"[14.0, 14.0, 16.0, 15.4, 14.0, 15.0, 14.0, 14....","[949536000.0, 956016000.0, 966816000.0, 977270...",-3.096703
14975,370120113555201,POINT (-113.93108 37.02164),[765.4],[1110844800.0],217.290970
14976,370211110185901,POINT (-110.31642 37.03642),"[13.0, 14.0, 14.0, 14.0, 15.1, 14.0, 14.0, 12....","[949536000.0, 956016000.0, 966816000.0, 966816...",1.250729


In [122]:
wells[wells["wellid"]==343957112322001]

,wellid,geometry,depth,utc_seconds,depth_20_year_trend
12747,343957112322001,POINT (-112.53878 34.66600),"[54.7, 52.6, 59.3, 63.0, 60.7, 56.2, 60.0, 67....","[951868800.0, 984700800.0, 1018483200.0, 10474...",3.783384


In [ ]:
wells.to_parquet("our_data/wells_with_depth_trend.parquet")